In [11]:
import numpy as np
import pandas as pd
import torch                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from dfg import check_dfg_compliance

def model_eval_test(dataset_name, csvwriter=None):
    '''
       This module is for validation and testing the Generator
       @param modelG: Generator neural network
       @param mode: 'validation', 'test', 'test-validation'
       @param obj: A data object created from "Input" class that contains the required information
       @return: The accuracy of the Generator
       '''
    # set the evaluation mode (this mode is necessary if you train with batch, since in test the size of batch is different)
    
    batch = 1
    #events = list(np.arange(0, len(obj.unique_event) + 1))
    predicted = []
    accuracy = []
    mae = []
    dfg_compliance_pred = []
    dfg_compliance_gt = []
    y_truth_list = []
    y_pred_last_event_list = []
    accuracy_last1 = []
    accuracy_last2 = []
    accuracy_last3 = []
    lastk = {}
    df = pd.read_pickle("dataset/preprocessed/"+dataset_name+"_design_mat.pkl")
    unique_event = [0] + sorted(df['class'].unique())
    events = list(np.arange(0, len(unique_event)))
    print("events",events)
    max_activities = len(unique_event)
    dur = 0
    dur_gt = 0
    num_overall_goal_satisfied = 0
    num_overall_goal_not_satisfied = 0
    num_overall_goal_satisfied_gt = 0
    num_overall_goal_not_satisfied_gt = 0
    row = []
    selected_columns = np.arange(0,max_activities+1)
    y_pred_last_event_list_prev = None
    group = df.groupby('CaseID')
    cur_prefix_len = 1
    prev_event = None
    max_prefix_len = 13
    pred_events = []
    thresh = 13.89
    if dataset_name == "helpdesk":
        thresh = 13.89 
        max_prefix_len = 13
    if dataset_name == "bpi_12_w":
        thresh = 18.28
        max_prefix_len = 73
    if dataset_name == "traffic_ss":
        thresh = 607.04
        max_prefix_len = 16

    for name,gr in group:
        gr = gr.reset_index(drop=True)
        new_row = [0] * gr.shape[1]
        gr.loc[gr.shape[0]] = new_row
        gr.iloc[gr.shape[0] - 1, gr.columns.get_loc('0')] = 1  # End of line is denoted by class 0
        temp = torch.tensor(gr.values, dtype=torch.float, requires_grad=False)
        temp_shifted = torch.tensor(gr[['duration_time','class']].values, dtype=torch.float, requires_grad=False)
        x = pad_sequence([temp], batch_first=True)
        y_truth = pad_sequence([temp_shifted], batch_first=True)
        cur_trace_len = int(x.shape[1])
        if cur_trace_len > max_prefix_len:
            continue
        activites = torch.argmax(x[:, :, events])
        # When we create mini batches, the length of the last one probably is less than the batch size, and it makes problem for the LSTM, therefore we skip it.
        if (x.size()[0] < batch):
            continue
        for cur_prefix_len in range(1,cur_trace_len):
            
            # Separating event and timestamp
            y_truth_timestamp = y_truth[:, cur_prefix_len, 0].view(batch, 1, -1)
            
            y_truth_event = y_truth[:, cur_prefix_len, 1].view(batch, 1, -1)
            
            # Executing LSTM
            if cur_prefix_len == 1:
                x_inn = x[:,  :cur_prefix_len, selected_columns]
                prev_event = x[:,  :cur_prefix_len, len(selected_columns)+1].ravel().detach().numpy().astype("int")
            else:
                x_inn = x[:,  :cur_prefix_len, selected_columns]
                
            rnnG = torch.load("checkpoints/"+dataset_name+"/event_timestamp_prediction/prefix_"+str(cur_prefix_len)+"/rnnG.m")
            rnnG.eval()
            y_pred = rnnG(x_inn[:,  :cur_prefix_len, selected_columns])

            # Just taking the last predicted element from each the batch
            y_pred_last = y_pred[0: batch, cur_prefix_len - 1, :]
            y_pred_last = y_pred_last.view((batch, 1, -1))
            y_pred_last_event = torch.argmax(F.softmax(y_pred_last[:, :, events], dim=2), dim=2)

            #Storing list of predictions and corresponding ground truths (to be used for f1score)
            y_truth_list += list(y_truth_event.flatten().data.cpu().numpy().astype(int))
            y_pred_last_event_list += list(y_pred_last_event.flatten().data.cpu().numpy().astype(int))
            
            # checking dfg compliance for predicted event
            if not(int(prev_event[0]) == 0 and int(y_pred_last_event.detach()[0][0])==0):
                dfg_compliance_bool = check_dfg_compliance(prev_event , y_pred_last_event.detach(), dset=dataset_name)
                dfg_compliance_pred.append(int(dfg_compliance_bool))

                dfg_compliance_gt_bool = check_dfg_compliance(prev_event , y_truth_event.detach().reshape(y_pred_last_event.shape), dset=dataset_name)
                dfg_compliance_gt.append(int(dfg_compliance_gt_bool))

                if y_pred_last_event.flatten().data.cpu().numpy().astype(int)==y_truth_event.flatten().data.cpu().numpy().astype(int):
                    accuracy.append(1)
                else:
                    accuracy.append(0)
                pred_events.append(int(y_pred_last_event.detach()[0][0]))

            # Computing MAE for the timestamp
            y_pred_timestamp = y_pred_last[:, :, len(events)].view((batch, 1, -1))
            mae.append(torch.abs(y_truth_timestamp - y_pred_timestamp).mean().detach())
            dur += max(y_pred_timestamp.detach().numpy()[0][0],0)  #adding to total proccess duration, making sure y_pred is non-negative
            dur_gt += max(y_truth_timestamp.detach().numpy()[0][0],0)
            prev_event = y_pred_last_event.ravel().detach().numpy().astype("int")
            
            if prev_event ==0:  #model predicted 0
                break
            
        # GS cases
        try:
            if dur < thresh:
                num_overall_goal_satisfied += 1
            else:
                num_overall_goal_not_satisfied += 1

            if dur_gt < thresh:
                num_overall_goal_satisfied_gt += 1
            else:
                num_overall_goal_not_satisfied_gt += 1
        except Exception as e: print(e)
#         print(pred_events)  
        lenn = min(5, len(pred_events))
        if lenn<=2:
            lenn = 3
        if len(pred_events)>=2:
            for i in range(2,lenn):
                if pred_events[-i] in lastk:
                    lastk[pred_events[-i]] += 1
                else:
                    lastk[pred_events[-i]] = 1
                if i == 1+1:
                    if(pred_events[-i] == y_truth_list[-i]): #action chosen by RL agent = gt
                        accuracy_last1.append(1)
                        accuracy_last2.append(1)
                        accuracy_last3.append(1)
                    else:
                        accuracy_last1.append(0)
                        accuracy_last2.append(0)
                        accuracy_last3.append(0)
                if i == 2+1:
                    if(pred_events[-i] == y_truth_list[-i]): #action chosen by RL agent = gt
                        accuracy_last2.append(1)
                        accuracy_last3.append(1)
                    else:
                        accuracy_last2.append(0)
                        accuracy_last3.append(0)
                if i == 3+1:
                    if(pred_events[-i] == y_truth_list[-i]): #action chosen by RL agent = gt
                        accuracy_last3.append(1)
                    else:
                        accuracy_last3.append(0)
        dur = 0
        dur_gt = 0
        pred_events = []
    tot = num_overall_goal_satisfied+ num_overall_goal_not_satisfied
    gs = num_overall_goal_satisfied/tot
    gv = num_overall_goal_not_satisfied/tot
    
    tot_gt = num_overall_goal_satisfied_gt+ num_overall_goal_not_satisfied_gt
    gs_gt = num_overall_goal_satisfied_gt/tot_gt
    gv_gt = num_overall_goal_not_satisfied_gt/tot_gt
    print("lastk",lastk)
    print("compliance pred", np.mean(np.array(dfg_compliance_pred)))
    print("compliance gt", np.mean(np.array(dfg_compliance_gt)))
    print("gs ", gs)
    print("gv ", gv)
    print("gs gt ", gs_gt)
    print("gv gt ", gv_gt)
    print("acc ",np.mean(np.array(accuracy)))
    print("acc last 3",np.mean(np.array(accuracy_last3)))
    print("acc last 2",np.mean(np.array(accuracy_last2)))
    print("acc last 1",np.mean(np.array(accuracy_last1)))
    print("mae ", np.mean(np.array(mae)))

In [12]:
datasets = ["helpdesk", "bpi_12_w","traffic_ss"]  
for dataset in datasets:
    print("dataset: ",dataset)
    model_eval_test(dataset_name=dataset, csvwriter=None)

dataset:  helpdesk
events [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
lastk {6: 5153, 8: 249}
compliance pred 0.8178766588602654
compliance gt 0.7641686182669789
gs  0.8550762756443977
gv  0.1449237243556023
gs gt  0.7498684902682798
gv gt  0.25013150973172016
acc  0.7395784543325526
acc last 3 0.6260644205849686
acc last 2 0.6821486268174475
acc last 1 0.8852631578947369
mae  3.3614337
dataset:  bpi_12_w
events [0, 1, 2, 3, 4, 5, 6]
lastk {5: 8661, 6: 621, 3: 5490, 4: 3670, 1: 264}
compliance pred 0.9331456938280006
compliance gt 0.8654717535530365
gs  0.7924821373097235
gv  0.20751786269027647
gs gt  0.7630734182458321
gv gt  0.23692658175416795
acc  0.6742031764687688
acc last 3 0.7146370148615417
acc last 2 0.6664921857474014
acc last 1 0.5734360221591677
mae  1.7013488
dataset:  traffic_ss
events [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
lastk {5: 7291, 4: 8613, 3: 7064, 2: 6634, 6: 951, 8: 348, 9: 98, 10: 76}
compliance pred 0.8214648399223218
compliance gt 0.91706604429083
gs  0.502760226139009

In [ ]:
# import numpy as np
# import pandas as pd
# import torch                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.nn.utils.rnn import pad_sequence
# from dfg import check_dfg_compliance

# def model_eval_test(dataset_name, csvwriter=None):
#     '''
#        This module is for validation and testing the Generator
#        @param modelG: Generator neural network
#        @param mode: 'validation', 'test', 'test-validation'
#        @param obj: A data object created from "Input" class that contains the required information
#        @return: The accuracy of the Generator
#        '''
#     # set the evaluation mode (this mode is necessary if you train with batch, since in test the size of batch is different)
    
#     batch = 1
#     #events = list(np.arange(0, len(obj.unique_event) + 1))
#     predicted = []
#     accuracy = []
#     mae = []
#     dfg_compliance_pred = []
#     dfg_compliance_gt = []
#     y_truth_list = []
#     y_pred_last_event_list = []
#     df = pd.read_pickle("dataset/preprocessed/"+dataset_name+"_design_mat.pkl")
#     unique_event = [0] + sorted(df['class'].unique())
#     events = list(np.arange(0, len(unique_event)))
#     print("events",events)
#     max_activities = len(unique_event)
#     dur = 0
#     dur_gt = 0
#     num_overall_goal_satisfied = 0
#     num_overall_goal_not_satisfied = 0
#     num_overall_goal_satisfied_gt = 0
#     num_overall_goal_not_satisfied_gt = 0
#     row = []
#     selected_columns = np.arange(0,max_activities+1)
#     y_pred_last_event_list_prev = None
#     group = df.groupby('CaseID')
#     cur_prefix_len = 1
#     prev_event = None
#     max_prefix_len = 13
#     thresh = 13.89
#     if dataset_name == "helpdesk":
#         thresh = 13.89 
#         max_prefix_len = 13
#     if dataset_name == "bpi_12_w":
#         thresh = 18.28
#         max_prefix_len = 73
#     if dataset_name == "traffic_ss":
#         thresh = 607.04
#         max_prefix_len = 16

#     for name,gr in group:
#         gr = gr.reset_index(drop=True)
#         new_row = [0] * gr.shape[1]
#         gr.loc[gr.shape[0]] = new_row
#         gr.iloc[gr.shape[0] - 1, gr.columns.get_loc('0')] = 1  # End of line is denoted by class 0
#         temp = torch.tensor(gr.values, dtype=torch.float, requires_grad=False)
#         temp_shifted = torch.tensor(gr[['duration_time','class']].values, dtype=torch.float, requires_grad=False)
#         x = pad_sequence([temp], batch_first=True)
#         y_truth = pad_sequence([temp_shifted], batch_first=True)
#         cur_trace_len = int(x.shape[1])
#         if cur_trace_len > max_prefix_len:
#             continue
#         activites = torch.argmax(x[:, :, events])
#         # When we create mini batches, the length of the last one probably is less than the batch size, and it makes problem for the LSTM, therefore we skip it.
#         if (x.size()[0] < batch):
#             continue
#         for cur_prefix_len in range(1,cur_trace_len):
            
#             # Separating event and timestamp
#             y_truth_timestamp = y_truth[:, cur_prefix_len, 0].view(batch, 1, -1)
            
#             y_truth_event = y_truth[:, cur_prefix_len, 1].view(batch, 1, -1)
            
#             # Executing LSTM
#             if cur_prefix_len == 1:
#                 x_inn = x[:,  :cur_prefix_len, selected_columns]
#                 prev_event = x[:,  :cur_prefix_len, len(selected_columns)+1].ravel().detach().numpy().astype("int")
#             else:
#                 x_inn = x[:,  :cur_prefix_len, selected_columns]
                
#             rnnG = torch.load("checkpoints/"+dataset_name+"/event_timestamp_prediction/prefix_"+str(cur_prefix_len)+"/rnnG.m")
#             rnnG.eval()
#             y_pred = rnnG(x_inn[:,  :cur_prefix_len, selected_columns])

#             # Just taking the last predicted element from each the batch
#             y_pred_last = y_pred[0: batch, cur_prefix_len - 1, :]
#             y_pred_last = y_pred_last.view((batch, 1, -1))
#             y_pred_last_event = torch.argmax(F.softmax(y_pred_last[:, :, events], dim=2), dim=2)

#             #Storing list of predictions and corresponding ground truths (to be used for f1score)
#             y_truth_list += list(y_truth_event.flatten().data.cpu().numpy().astype(int))
#             y_pred_last_event_list += list(y_pred_last_event.flatten().data.cpu().numpy().astype(int))

#             # checking dfg compliance for predicted event
#             if not(int(prev_event[0]) == 0 and int(y_pred_last_event.detach()[0][0])==0):
#                 dfg_compliance_bool = check_dfg_compliance(prev_event , y_pred_last_event.detach(), dset=dataset_name)
#                 dfg_compliance_pred.append(int(dfg_compliance_bool))

#                 dfg_compliance_gt_bool = check_dfg_compliance(prev_event , y_truth_event.detach().reshape(y_pred_last_event.shape), dset=dataset_name)
#                 dfg_compliance_gt.append(int(dfg_compliance_gt_bool))

#                 if y_pred_last_event.flatten().data.cpu().numpy().astype(int)==y_truth_event.flatten().data.cpu().numpy().astype(int):
#                     accuracy.append(1)
#                 else:
#                     accuracy.append(0)

#             # Computing MAE for the timestamp
#             y_pred_timestamp = y_pred_last[:, :, len(events)].view((batch, 1, -1))
#             mae.append(torch.abs(y_truth_timestamp - y_pred_timestamp).mean().detach())
#             dur += max(y_pred_timestamp.detach().numpy()[0][0],0)  #adding to total proccess duration, making sure y_pred is non-negative
#             dur_gt += max(y_truth_timestamp.detach().numpy()[0][0],0)
#             prev_event = y_pred_last_event.ravel().detach().numpy().astype("int")
            
#         # GS cases
#         try:
#             if dur < thresh:
#                 num_overall_goal_satisfied += 1
#             else:
#                 num_overall_goal_not_satisfied += 1

#             if dur_gt < thresh:
#                 num_overall_goal_satisfied_gt += 1
#             else:
#                 num_overall_goal_not_satisfied_gt += 1
#         except Exception as e: print(e)
#         dur = 0
#         dur_gt = 0
#     tot = num_overall_goal_satisfied+ num_overall_goal_not_satisfied
#     gs = num_overall_goal_satisfied/tot
#     gv = num_overall_goal_not_satisfied/tot
    
#     tot_gt = num_overall_goal_satisfied_gt+ num_overall_goal_not_satisfied_gt
#     gs_gt = num_overall_goal_satisfied_gt/tot_gt
#     gv_gt = num_overall_goal_not_satisfied_gt/tot_gt
#     print("compliance pred", np.mean(np.array(dfg_compliance_pred)))
#     print("compliance gt", np.mean(np.array(dfg_compliance_gt)))
#     print("gs ", gs)
#     print("gv ", gv)
#     print("gs gt", gs_gt)
#     print("gv gt", gv_gt)
#     print("acc ",np.mean(np.array(accuracy)))
#     print("mae ", np.mean(np.array(mae)))
# datasets = ["traffic_ss"]  

# for dataset in datasets:
#     print("dataset: ",dataset)
#     model_eval_test(dataset_name=dataset, csvwriter=None)